# Drug Classification — Featureless Model (Speed + Turning Angle only)

Objectif : prédire si un worm est **drugged** ou **control** à partir de segments
de mouvement bruts, en utilisant uniquement **Speed** et **turning_angle**.

On enlève volontairement X et Y pour éviter que le modèle exploite la position
sur la plaque (les worms drogués étant spatialement groupés).


In [4]:
import os
import glob

import numpy as np
import pandas as pd

from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier  # si pas installé: pip install xgboost

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6, 4)

# Dossier contenant les segments pré-traités
SEGMENTS_DIR = "preprocessed_data/segments"

# Longueur d'un segment (en frames)
SEGMENT_LENGTH = 900

# Features utilisées (SANS X, Y)
FEATURE_COLS = ["Speed", "turning_angle"]

# Noms de colonnes dans les CSV de segments pré-traités
TIME_COL        = "GlobalFrame"
SOURCE_FILE_COL = "source_file"     # identifiant du worm
SEGMENT_IDX_COL = "Segment_index"
CONDITION_COL   = "condition"       # contient l'info drug/control



inspection rapide fichier de segment

In [5]:
files = sorted(glob.glob(os.path.join(SEGMENTS_DIR, "*.csv")))
print("Nb de fichiers de segments trouvés:", len(files))
print("Exemples:")
for f in files[:5]:
    print(" -", os.path.basename(f))

# Inspecter un fichier pour vérifier les colonnes
df_example = pd.read_csv(files[0])
print("\nColonnes d'un segment d'exemple:")
print(df_example.columns.tolist())
df_example.head()



Nb de fichiers de segments trouvés: 8150
Exemples:
 - coordinates_highestspeed_20240827_10_1_with_time_speed-fragment0.0-preprocessed.csv
 - coordinates_highestspeed_20240827_10_1_with_time_speed-fragment1.0-preprocessed.csv
 - coordinates_highestspeed_20240827_10_1_with_time_speed-fragment10.0-preprocessed.csv
 - coordinates_highestspeed_20240827_10_1_with_time_speed-fragment11.0-preprocessed.csv
 - coordinates_highestspeed_20240827_10_1_with_time_speed-fragment12.0-preprocessed.csv

Colonnes d'un segment d'exemple:
['GlobalFrame', 'Timestamp', 'Speed', 'X', 'Y', 'condition', 'source_file', 'Segment_index', 'turning_angle', 'worm_id', 'Segment']


,GlobalFrame,Timestamp,Speed,X,Y,condition,source_file,Segment_index,turning_angle,worm_id,Segment
0,2.0,2024-08-27T15:43:40.009947,-0.424172,0.415679,0.405714,control,coordinates_highestspeed_20240827_10_1_with_ti...,0.0,0.000000,20240827_piworm10_1,0.0
1,3.0,2024-08-27T15:43:42.001966,-0.424172,0.416102,0.405726,control,coordinates_highestspeed_20240827_10_1_with_ti...,0.0,-0.215921,20240827_piworm10_1,0.0
2,4.0,2024-08-27T15:43:44.002485,-0.495096,0.416319,0.405562,control,coordinates_highestspeed_20240827_10_1_with_ti...,0.0,-0.200451,20240827_piworm10_1,0.0
3,5.0,2024-08-27T15:43:46.002854,-0.401777,0.416455,0.405110,control,coordinates_highestspeed_20240827_10_1_with_ti...,0.0,0.198548,20240827_piworm10_1,0.0
4,6.0,2024-08-27T15:43:48.002383,-0.512398,0.416641,0.404967,control,coordinates_highestspeed_20240827_10_1_with_ti...,0.0,-0.173521,20240827_piworm10_1,0.0


loader

In [6]:
def load_segment_file(path):
    """
    Charge un fichier de segment pré-traité et renvoie :
      - raw : array (T, len(FEATURE_COLS))
      - worm_id : identifiant du worm (source_file)
      - segment_index : index de segment
      - condition : "control", "terb", etc.
    """
    df = pd.read_csv(path)

    # vérification des colonnes nécessaires
    required = [TIME_COL, SOURCE_FILE_COL, SEGMENT_IDX_COL, CONDITION_COL] + FEATURE_COLS
    for c in required:
        if c not in df.columns:
            raise RuntimeError(f"Column '{c}' missing in {os.path.basename(path)}")

    # tri temporel
    df = df.sort_values(TIME_COL)

    worm_id = str(df[SOURCE_FILE_COL].iloc[0])
    seg_idx = int(df[SEGMENT_IDX_COL].iloc[0])
    condition = str(df[CONDITION_COL].iloc[0]).lower()

    raw = df[FEATURE_COLS].to_numpy()

    # pad / tronque à SEGMENT_LENGTH frames
    if raw.shape[0] < SEGMENT_LENGTH:
        pad_len = SEGMENT_LENGTH - raw.shape[0]
        pad = np.tile(raw[-1:, :], (pad_len, 1))
        raw = np.vstack([raw, pad])
    elif raw.shape[0] > SEGMENT_LENGTH:
        raw = raw[:SEGMENT_LENGTH, :]

    return raw, worm_id, seg_idx, condition

print("FEATURE_COLS =", FEATURE_COLS)



FEATURE_COLS = ['Speed', 'turning_angle']


charger tous segments et metadonnes

In [7]:
segment_files = sorted(glob.glob(os.path.join(SEGMENTS_DIR, "*.csv")))
print("Nb de segments:", len(segment_files))

all_features = []
meta_rows = []

for path in segment_files:
    feats, worm_id, seg_idx, cond = load_segment_file(path)
    all_features.append(feats)
    meta_rows.append({
        "filename": os.path.basename(path),
        "worm_id": worm_id,
        "segment_index": seg_idx,
        "condition": cond,
    })

meta_df = pd.DataFrame(meta_rows)
print(meta_df.head())
print("Nb de worms différents:", meta_df["worm_id"].nunique())



Nb de segments: 8150
                                            filename  \
0  coordinates_highestspeed_20240827_10_1_with_ti...   
1  coordinates_highestspeed_20240827_10_1_with_ti...   
2  coordinates_highestspeed_20240827_10_1_with_ti...   
3  coordinates_highestspeed_20240827_10_1_with_ti...   
4  coordinates_highestspeed_20240827_10_1_with_ti...   

                                             worm_id  segment_index condition  
0  coordinates_highestspeed_20240827_10_1_with_ti...              0   control  
1  coordinates_highestspeed_20240827_10_1_with_ti...              1   control  
2  coordinates_highestspeed_20240827_10_1_with_ti...             10   control  
3  coordinates_highestspeed_20240827_10_1_with_ti...             11   control  
4  coordinates_highestspeed_20240827_10_1_with_ti...             12   control  
Nb de worms différents: 104


cinstruction label drugged/control 

In [8]:
print("Conditions disponibles:")
print(meta_df["condition"].value_counts())

# Adapter ce mapping si besoin :
def condition_to_label(c: str) -> int:
    c = c.lower()
    # met ici tous les patterns qui veulent dire "drugged"
    if "terb" in c or "drug" in c or "treated" in c:
        return 1
    else:
        return 0  # control

meta_df["y"] = meta_df["condition"].apply(condition_to_label)
y = meta_df["y"].values
groups = meta_df["worm_id"].values

print("Répartition des labels (0=control, 1=drugged):", np.bincount(y))


Conditions disponibles:
condition
terbinafine    4076
control        4074
Name: count, dtype: int64
Répartition des labels (0=control, 1=drugged): [4074 4076]


construction X_raw et X_flat

In [9]:
X_raw = np.stack(all_features)   # (N, 900, 2)
N, T, F = X_raw.shape
print("X_raw shape:", X_raw.shape)

X_flat = X_raw.reshape(N, T * F)  # (N, 1800)
print("X_flat shape:", X_flat.shape)


X_raw shape: (8150, 900, 2)
X_flat shape: (8150, 1800)


split train/val/test groupe par worm

In [10]:
from sklearn.model_selection import GroupShuffleSplit

# 70% train, 30% temp
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, temp_idx = next(gss1.split(X_flat, y, groups))

# temp -> 15% val, 15% test
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=43)
val_idx_temp, test_idx_temp = next(gss2.split(X_flat[temp_idx], y[temp_idx], groups[temp_idx]))

val_idx = temp_idx[val_idx_temp]
test_idx = temp_idx[test_idx_temp]

print("Train / Val / Test:", len(train_idx), len(val_idx), len(test_idx))
print("Classes train:", np.bincount(y[train_idx]))
print("Classes val  :", np.bincount(y[val_idx]))
print("Classes test :", np.bincount(y[test_idx]))

X_train, X_val, X_test = X_flat[train_idx], X_flat[val_idx], X_flat[test_idx]
y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_val_sc   = scaler.transform(X_val)
X_test_sc  = scaler.transform(X_test)


Train / Val / Test: 5726 1275 1149
Classes train: [3044 2682]
Classes val  : [574 701]
Classes test : [456 693]


baseline gradient boosting (long a run= )

In [11]:
gb = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.08,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

%time gb.fit(X_train_sc, y_train)

# Validation
proba_val = gb.predict_proba(X_val_sc)[:, 1]
pred_val  = (proba_val >= 0.5).astype(int)

acc_val = accuracy_score(y_val, pred_val)
f1_val  = f1_score(y_val, pred_val)
auc_val = roc_auc_score(y_val, proba_val)

print("=== Validation performance (threshold=0.5) ===")
print(f"ACC={acc_val:.3f} | F1={f1_val:.3f} | AUC={auc_val:.3f}")

# Test
proba_test = gb.predict_proba(X_test_sc)[:, 1]
pred_test  = (proba_test >= 0.5).astype(int)

acc_test = accuracy_score(y_test, pred_test)
f1_test  = f1_score(y_test, pred_test)
auc_test = roc_auc_score(y_test, proba_test)

print("\n=== Test performance (threshold=0.5) ===")
print(f"ACC={acc_test:.3f} | F1={f1_test:.3f} | AUC={auc_test:.3f}")


CPU times: user 5min 38s, sys: 1.11 s, total: 5min 39s
Wall time: 5min 40s
=== Validation performance (threshold=0.5) ===
ACC=0.467 | F1=0.408 | AUC=0.470

=== Test performance (threshold=0.5) ===
ACC=0.420 | F1=0.439 | AUC=0.409


cross validation groupee (5folds)

In [12]:
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

clf = Pipeline([
    ("scaler", StandardScaler()),
    ("gb", GradientBoostingClassifier(
        n_estimators=150,
        learning_rate=0.08,
        max_depth=3,
        subsample=0.8,
        random_state=42
    ))
])

accs, f1s, aucs = [], [], []

for fold, (tr, te) in enumerate(cv.split(X_flat, y, groups)):
    print(f"\n=== Fold {fold+1} ===")
    clf.fit(X_flat[tr], y[tr])
    proba = clf.predict_proba(X_flat[te])[:, 1]
    preds = (proba >= 0.5).astype(int)

    accs.append(accuracy_score(y[te], preds))
    f1s.append(f1_score(y[te], preds))
    aucs.append(roc_auc_score(y[te], proba))

    print(f"ACC={accs[-1]:.3f} | F1={f1s[-1]:.3f} | AUC={aucs[-1]:.3f}")

print("\n=== CV summary (featureless drug classifier) ===")
print(f"ACC  : {np.mean(accs):.3f} ± {np.std(accs):.3f}")
print(f"F1   : {np.mean(f1s):.3f} ± {np.std(f1s):.3f}")
print(f"AUC  : {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")



=== Fold 1 ===
ACC=0.538 | F1=0.539 | AUC=0.556

=== Fold 2 ===
ACC=0.508 | F1=0.494 | AUC=0.515

=== Fold 3 ===
ACC=0.479 | F1=0.417 | AUC=0.476

=== Fold 4 ===
ACC=0.506 | F1=0.497 | AUC=0.548

=== Fold 5 ===
ACC=0.402 | F1=0.420 | AUC=0.509

=== CV summary (featureless drug classifier) ===
ACC  : 0.487 ± 0.046
F1   : 0.473 ± 0.048
AUC  : 0.521 ± 0.029


model comparison (GB/LogREG/LinearSVM/XGBoost)

In [ ]:
print("=== Model comparison (featureless, Speed + turning_angle) ===")

# Nouveau split pour comparaison modèles
gss_mc = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=123)
train_idx_mc, test_idx_mc = next(gss_mc.split(X_flat, y, groups))

X_train_mc, X_test_mc = X_flat[train_idx_mc], X_flat[test_idx_mc]
y_train_mc, y_test_mc = y[train_idx_mc], y[test_idx_mc]

scaler_mc = StandardScaler()
X_train_mc_sc = scaler_mc.fit_transform(X_train_mc)
X_test_mc_sc  = scaler_mc.transform(X_test_mc)

models = {
    "GradientBoosting": GradientBoostingClassifier(
        n_estimators=150,
        learning_rate=0.08,
        max_depth=3,
        subsample=0.8,
        random_state=42
    ),
    "LogisticRegression": LogisticRegression(
        max_iter=500,
        class_weight="balanced",
        random_state=42
    ),
    "LinearSVM": CalibratedClassifierCV(
        LinearSVC(C=1.0, class_weight="balanced", random_state=42),
        method="sigmoid",
        cv=3
    ),
    "XGBoost": XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42
    ),
}

results_models = []

for name, model in models.items():
    print(f"\n--- {name} ---")
    model.fit(X_train_mc_sc, y_train_mc)
    proba = model.predict_proba(X_test_mc_sc)[:, 1]
    preds = (proba >= 0.5).astype(int)

    acc = accuracy_score(y_test_mc, preds)
    f1  = f1_score(y_test_mc, preds)
    auc = roc_auc_score(y_test_mc, proba)

    print(f"ACC={acc:.3f} | F1={f1:.3f} | AUC={auc:.3f}")

    results_models.append({"model": name, "ACC": acc, "F1": f1, "AUC": auc})

results_models_df = pd.DataFrame(results_models)
results_models_df
